In [1]:
import pathlib as pl
import pandas as pd
import re

pd.set_option('future.no_silent_downcasting', True)

%cd -q "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/notebooks"
_PROJECT_CONFIG_NB = str(pl.Path("00_project_config.ipynb").resolve(strict=True))

%run $_PROJECT_CONFIG_NB

_MYNAME="asm-stats-summary-table"
_NBSTAMP=get_nb_stamp(_MYNAME)

assembler = "verkko"
assembler_versions = {
    "verkko": "Verkko v1.4.1",
    "hifiasm": "hifiasm v0.19.5"
}

input_path = PROJECT_DATA_ROOT.joinpath(
    f"2023_assm_stats/{assembler}/hgsvc3/contig_stats"
)

output_table_sheet = PROJECT_BASE.joinpath(
    "annotations", "autogen", f"{assembler}_assemblies.hgsvc3.tsv"
)

merged = []
for tsv_file in input_path.glob("*.summary.tsv"):
    name_parts = tsv_file.name.split(".")
    sample = name_parts[0]
    sample_num = int(HGSVC_SAMPLES.loc[HGSVC_SAMPLES["sample"] == sample, "order_num"].values[0])
    sample_batch = int(HGSVC_SAMPLES.loc[HGSVC_SAMPLES["sample"] == sample, "batch_num"].values[0])
    asm_unit = name_parts[2]
    df = pd.read_csv(tsv_file, sep="\t", header=0)
    if df.empty:
        df = pd.DataFrame(
            [
                ["all", "total_length_grt_0bp", 0, sample, sample_num, sample_batch, asm_unit],
                ["all", "total_num_grt_0bp", 0, sample, sample_num, sample_batch, asm_unit]
            ],
            columns=["source", "statistic", "value", "sample", "sample_num", "verkko_batch", "sequence"]
        )
    else:
        df["sample"] = sample
        df["sample_num"] = sample_num
        df["verkko_batch"] = sample_batch
        df["sequence"] = asm_unit
    merged.append(df)
    
merged = pd.concat(merged, axis=0, ignore_index=False)
merged.drop("source", axis=1, inplace=True)
row_index = pd.MultiIndex.from_tuples(
    sorted(set([(row.sample, int(row.sample_num), int(row.verkko_batch)) for row in merged.itertuples()])),
    names=["sample", "sample_num", "verkko_batch"]
)
column_index = pd.Index(
    [(seq, stat) for (seq, stat), _ in merged.groupby(["sequence", "statistic"])],
    tupleize_cols=True, name=("sequence", "statistic"))

pivot = pd.DataFrame([], index=row_index, columns=column_index)
for row in merged.itertuples():
    pivot.loc[(row.sample, row.sample_num), (row.sequence, row.statistic)] = row.value
pivot = pivot.fillna(0, inplace=False).infer_objects(copy=False)

sort_seq = {
    'asm-hap1': 0,
    'asm-hap2': 1,
    'asm-unassigned': 2,
    'asm-rdna': 3,
    'asm-disconnected': 4,
    'contaminants': 5
}

if assembler == "verkko":

    aun = "length_auN_grt_0bp"
    length = "total_length_grt_0bp"
       
    for other_seq in ["asm-unassigned", "asm-disconnected"]:
    
        other_length = pivot.xs((other_seq, length), level=["sequence", "statistic"], axis=1)
        hap1_length = pivot.xs(("asm-hap1", length), level=["sequence", "statistic"], axis=1).values
        hap2_length = pivot.xs(("asm-hap2", length), level=["sequence", "statistic"], axis=1).values
        dip_length = hap1_length + hap2_length
        pct_length = ((other_length.values / dip_length) * 100).round(3)
        pct_length = pd.DataFrame(
            pct_length,
            index=pivot.index,
            columns=pd.MultiIndex.from_tuples(
                [(other_seq, "pct_dip_length_grt_0bp")],
                names=["sequence", "statistic"]
            )
        )
        pivot = pivot.merge(pct_length, left_index=True, right_index=True)
pivot.sort_index(axis=0, level="sample_num", inplace=True)


def sort_column_index(column_index):
    
    stats_values = column_index.get_level_values("statistic")
    
    get_seq_context = re.compile("(0bp|[0-9]+[kM]bp)")
    sort_stat = {}
    for entry in set(stats_values):
        mobj = get_seq_context.search(entry)
        if mobj is None:
            raise ValueError(entry)
        seq_context = mobj.group(0)
        if "kbp" in seq_context:
            factor = 1e3
            order_num = int(int(seq_context.strip("kbp")) * factor)
        elif "Mbp" in seq_context:
            factor = 1e6
            order_num = int(int(seq_context.strip("Mbp")) * factor)
        else:
            order_num = 0
        sort_stat[entry] = order_num
    
    index_tuples = column_index.to_flat_index()
    
    sorted_tuples = sorted(
        [(sort_seq[t[0]], sort_stat[t[1]], t) for t in index_tuples]
    )
    
    sorted_mindex = pd.MultiIndex.from_tuples(
        [t[2] for t in sorted_tuples],
        names=["sequence", "statistic"]
    )
    
    return sorted_mindex

sorted_mindex_cols = sort_column_index(pivot.columns)

pivot = pivot.loc[:, sorted_mindex_cols]

with open(output_table_sheet, "w") as table_out:
    _ = table_out.write("# AUTOGEN TABLE - DO NOT EDIT\n")
    _ = table_out.write(f"# {_NBSTAMP}\n")
    _ = table_out.write(f"# {assembler_versions[assembler]} HGSVC3 phased assemblies\n")
    _ = table_out.write(f"# {pivot.index.get_level_values('sample').nunique()} samples\n")
    pivot.to_csv(table_out, sep="\t", index=True, header=True)